In [1]:
pip install feature-engine

  Obtaining dependency information for feature-engine from https://files.pythonhosted.org/packages/62/60/77fcc9d3cfaabab34027aa8ea0025c5e2d4cf9561fa9725a38f0785b43aa/feature_engine-1.6.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 10.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os 
from feature_engine.encoding import OrdinalEncoder
from feature_engine.outliers import OutlierTrimmer
from feature_engine.transformation import PowerTransformer
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , StackingClassifier , VotingClassifier
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [3]:
os.listdir("/kaggle/input/paysim1")

['PS_20174392719_1491204439457_log.csv']

In [4]:
data = pd.read_csv("/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv")
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [6]:
data.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [7]:
(data["isFraud"]== 1).sum()

8213

In [8]:
ode = OrdinalEncoder(encoding_method='arbitrary')
ode.fit(data)
data = ode.transform(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            int64  
 2   amount          float64
 3   nameOrig        int64  
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        int64  
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(6)
memory usage: 534.0 MB


In [9]:
data.head()
data.drop(columns = "step" , axis = 1 , inplace = True)

In [10]:
new_data = data.copy()
new_data.drop(columns= "isFlaggedFraud",axis = 1,inplace = True)
new_data.head()

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,0,9839.64,0,170136.0,160296.36,0,0.0,0.0,0
1,0,1864.28,1,21249.0,19384.72,1,0.0,0.0,0
2,1,181.00,2,181.0,0.00,2,0.0,0.0,1
3,2,181.00,3,181.0,0.00,3,21182.0,0.0,1
4,0,11668.14,4,41554.0,29885.86,4,0.0,0.0,0


In [11]:
ot = PowerTransformer()
ot.fit(new_data)
updated_data = ot.transform(new_data)
updated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   type            float64
 1   amount          float64
 2   nameOrig        float64
 3   oldbalanceOrg   float64
 4   newbalanceOrig  float64
 5   nameDest        float64
 6   oldbalanceDest  float64
 7   newbalanceDest  float64
 8   isFraud         int64  
dtypes: float64(8), int64(1)
memory usage: 436.9 MB


In [12]:
x_col = updated_data.copy()
x_col.drop(columns = "isFraud" , axis = 1,inplace = True)

In [13]:
x = updated_data[x_col.columns]
y = updated_data["isFraud"]

In [14]:
x_train , x_test, y_train , y_test = train_test_split(x,y, train_size= 0.7 , random_state= 32)

In [15]:
def Logisticregression(trial_lr):
    penalty_lr = trial_lr.suggest_categorical('penalty', ['l1', 'l2'])
    C = trial_lr.suggest_loguniform('C', 1e-5, 1e5)
    solver = trial_lr.suggest_categorical('solver', ['liblinear', 'saga'])
    max_iter = trial_lr.suggest_int('max_iter', 100, 1000)
    model_lr = LogisticRegression(penalty=penalty_lr,
        C=C,
        solver=solver,
        max_iter=max_iter)
    model_lr.fit(x_train, y_train)
    y_pred = model_lr.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(Logisticregression, n_trials=5, show_progress_bar=True)

best_params_lr = study_lr.best_params
best_accuracy_lr = study_lr.best_value
print(f"Best parameters: {best_params_lr}")
print(f"Best accuracy: {best_accuracy_lr}")

[I 2023-12-31 05:09:39,630] A new study created in memory with name: no-name-ca63cc77-b8b8-4ef8-b968-97eb33e59bb7


  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_42/1564662201.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial_lr.suggest_loguniform('C', 1e-5, 1e5)
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/tmp/ipykernel_42/1564662201.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial_lr.suggest_loguniform('C', 1e-5, 1e5)


[I 2023-12-31 05:34:06,154] Trial 0 finished with value: 0.9990229391875255 and parameters: {'penalty': 'l2', 'C': 0.6484178381569493, 'solver': 'saga', 'max_iter': 555}. Best is trial 0 with value: 0.9990229391875255.
[I 2023-12-31 05:34:47,068] Trial 1 finished with value: 0.9992723123493152 and parameters: {'penalty': 'l2', 'C': 0.0011052566345768058, 'solver': 'liblinear', 'max_iter': 533}. Best is trial 1 with value: 0.9992723123493152.


/tmp/ipykernel_42/1564662201.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial_lr.suggest_loguniform('C', 1e-5, 1e5)


[I 2023-12-31 06:02:20,833] Trial 2 finished with value: 0.9990229391875255 and parameters: {'penalty': 'l2', 'C': 36.564853778879986, 'solver': 'saga', 'max_iter': 761}. Best is trial 1 with value: 0.9992723123493152.


/tmp/ipykernel_42/1564662201.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial_lr.suggest_loguniform('C', 1e-5, 1e5)


[I 2023-12-31 06:29:55,236] Trial 3 finished with value: 0.9990229391875255 and parameters: {'penalty': 'l2', 'C': 0.008186230798733504, 'solver': 'saga', 'max_iter': 703}. Best is trial 1 with value: 0.9992723123493152.


/tmp/ipykernel_42/1564662201.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial_lr.suggest_loguniform('C', 1e-5, 1e5)


[I 2023-12-31 06:57:31,975] Trial 4 finished with value: 0.9990229391875255 and parameters: {'penalty': 'l2', 'C': 20.703928268760638, 'solver': 'saga', 'max_iter': 954}. Best is trial 1 with value: 0.9992723123493152.
Best parameters: {'penalty': 'l2', 'C': 0.0011052566345768058, 'solver': 'liblinear', 'max_iter': 533}
Best accuracy: 0.9992723123493152


In [16]:
def decision_tree(trial_dt):
    criterion = trial_dt.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial_dt.suggest_int('max_depth', 2, 32)
    min_samples_split = trial_dt.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial_dt.suggest_int('min_samples_leaf', 1, 20)


    dtc = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    dtc.fit(x_train, y_train)
    y_pred = dtc.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(decision_tree, n_trials=5,show_progress_bar= True)
best_params_dt = study_dt.best_params
best_accuracy_dt = study_dt.best_value
print("Best Hyperparameters:", best_params_dt)
print("Best Accuracy:", best_accuracy_dt)

[I 2023-12-31 06:57:31,992] A new study created in memory with name: no-name-4f96f9c6-f513-4793-966b-c329cbde8ba6


  0%|          | 0/5 [00:00<?, ?it/s]

[I 2023-12-31 06:59:10,461] Trial 0 finished with value: 0.9996594694219257 and parameters: {'criterion': 'gini', 'max_depth': 18, 'min_samples_split': 5, 'min_samples_leaf': 18}. Best is trial 0 with value: 0.9996594694219257.
[I 2023-12-31 07:00:13,999] Trial 1 finished with value: 0.9993000786887581 and parameters: {'criterion': 'gini', 'max_depth': 7, 'min_samples_split': 12, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9996594694219257.
[I 2023-12-31 07:01:19,167] Trial 2 finished with value: 0.9996615649947139 and parameters: {'criterion': 'entropy', 'max_depth': 23, 'min_samples_split': 7, 'min_samples_leaf': 19}. Best is trial 2 with value: 0.9996615649947139.
[I 2023-12-31 07:02:24,008] Trial 3 finished with value: 0.9996945702661273 and parameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 16, 'min_samples_leaf': 7}. Best is trial 3 with value: 0.9996945702661273.
[I 2023-12-31 07:03:28,712] Trial 4 finished with value: 0.9997123826348265 and par

In [17]:
def random_forest(trial_rf):
    criterion = trial_rf.suggest_categorical('criterion', ['gini', 'entropy'])
    n_estimators = trial_rf.suggest_int('n_estimators', 50, 200)
    max_depth = trial_rf.suggest_int('max_depth', 2, 32)
    min_samples_split = trial_rf.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial_rf.suggest_int('min_samples_leaf', 1, 20)

    rf = RandomForestClassifier(
        criterion=criterion,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42 
    )
    rf.fit(x_train, y_train)
    y_pred = rf.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(random_forest, n_trials=5, show_progress_bar=True)

best_params_rf = study_rf.best_params
best_accuracy_rf = study_rf.best_value
print("Best Hyperparameters:", best_params_rf)
print("Best Accuracy:", best_accuracy_rf)

[I 2023-12-31 07:03:28,729] A new study created in memory with name: no-name-4d226329-b296-416f-ae7c-c0f868a555fe


  0%|          | 0/5 [00:00<?, ?it/s]

[I 2023-12-31 07:18:56,528] Trial 0 finished with value: 0.9987824722100853 and parameters: {'criterion': 'gini', 'n_estimators': 156, 'max_depth': 3, 'min_samples_split': 9, 'min_samples_leaf': 19}. Best is trial 0 with value: 0.9987824722100853.
[I 2023-12-31 07:29:31,536] Trial 1 finished with value: 0.9995892677335227 and parameters: {'criterion': 'entropy', 'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 15, 'min_samples_leaf': 15}. Best is trial 1 with value: 0.9995892677335227.
[I 2023-12-31 07:53:14,246] Trial 2 finished with value: 0.9993645175519937 and parameters: {'criterion': 'entropy', 'n_estimators': 129, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 19}. Best is trial 1 with value: 0.9995892677335227.
[I 2023-12-31 08:27:34,425] Trial 3 finished with value: 0.999676757897428 and parameters: {'criterion': 'gini', 'n_estimators': 106, 'max_depth': 25, 'min_samples_split': 20, 'min_samples_leaf': 3}. Best is trial 3 with value: 0.999676757897428.
[I

In [18]:
def xgboost(trial_xgb):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'verbosity': 0,
        'tree_method': 'auto'
    }

    max_depth = trial_xgb.suggest_int('max_depth', 3, 10)
    learning_rate = trial_xgb.suggest_loguniform('learning_rate', 0.005, 0.5)
    subsample = trial_xgb.suggest_float('subsample', 0.6, 1.0)
    colsample_bytree = trial_xgb.suggest_float('colsample_bytree', 0.6, 1.0)
    min_child_weight = trial_xgb.suggest_int('min_child_weight', 1, 10)
    gamma = trial_xgb.suggest_float('gamma', 0, 10)
    model = XGBClassifier(max_depth = max_depth,
                          learning_rate = learning_rate,
                          subsample = subsample,
                          colsample_bytree = colsample_bytree,
                          min_child_weight = min_child_weight,
                          gamma = gamma)
    model.fit(x_train,y_train)

    preds = model.predict(x_test)
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(xgboost, n_trials=5, show_progress_bar=True)

best_params_xgb = study_xgb.best_params
best_accuracy_xgb = study_xgb.best_value
print("Best Hyperparameters:", best_params_xgb)
print("Best Accuracy:", best_accuracy_xgb)

[I 2023-12-31 08:51:41,845] A new study created in memory with name: no-name-885a06c5-81eb-4b54-b6d2-0740d89ebe31


  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_42/1123641493.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial_xgb.suggest_loguniform('learning_rate', 0.005, 0.5)


[I 2023-12-31 08:52:02,370] Trial 0 finished with value: 0.9995997455974636 and parameters: {'max_depth': 9, 'learning_rate': 0.024382680823835295, 'subsample': 0.6143054311483256, 'colsample_bytree': 0.9417781704365737, 'min_child_weight': 4, 'gamma': 5.403888857851841}. Best is trial 0 with value: 0.9995997455974636.


/tmp/ipykernel_42/1123641493.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial_xgb.suggest_loguniform('learning_rate', 0.005, 0.5)


...

[I 2023-12-31 08:52:40,024] Trial 2 finished with value: 0.9995279722294694 and parameters: {'max_depth': 7, 'learning_rate': 0.02400246394486078, 'subsample': 0.7465279230705056, 'colsample_bytree': 0.874341457864634, 'min_child_weight': 5, 'gamma': 9.56117172266008}. Best is trial 0 with value: 0.9995997455974636.


/tmp/ipykernel_42/1123641493.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial_xgb.suggest_loguniform('learning_rate', 0.005, 0.5)


[I 2023-12-31 08:53:01,023] Trial 3 finished with value: 0.9995635969668679 and parameters: {'max_depth': 5, 'learning_rate': 0.05089765133765627, 'subsample': 0.6437678707216398, 'colsample_bytree': 0.6288732654743389, 'min_child_weight': 9, 'gamma': 3.649813800849424}. Best is trial 0 with value: 0.9995997455974636.


/tmp/ipykernel_42/1123641493.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial_xgb.suggest_loguniform('learning_rate', 0.005, 0.5)


[I 2023-12-31 08:53:19,977] Trial 4 finished with value: 0.9993047937275316 and parameters: {'max_depth': 5, 'learning_rate': 0.014437129493915159, 'subsample': 0.9407936801740401, 'colsample_bytree': 0.9499547536633766, 'min_child_weight': 2, 'gamma': 1.9508689170758353}. Best is trial 0 with value: 0.9995997455974636.
Best Hyperparameters: {'max_depth': 9, 'learning_rate': 0.024382680823835295, 'subsample': 0.6143054311483256, 'colsample_bytree': 0.9417781704365737, 'min_child_weight': 4, 'gamma': 5.403888857851841}
Best Accuracy: 0.9995997455974636


In [19]:
lr = LogisticRegression(penalty= best_params_lr['penalty'],C = best_params_lr['C'],solver= best_params_lr['solver'],max_iter= best_params_lr['max_iter'])
dt = DecisionTreeClassifier(criterion= best_params_dt['criterion'], max_depth= best_params_dt['max_depth'], min_samples_split= best_params_dt['min_samples_split'], min_samples_leaf= best_params_dt['min_samples_leaf'])
rf = RandomForestClassifier(criterion= best_params_rf['criterion'],n_estimators= best_params_rf['n_estimators'], max_depth= best_params_rf['max_depth'],min_samples_leaf= best_params_rf['min_samples_leaf'], min_samples_split= best_params_rf['min_samples_split'])
xgb = XGBClassifier(max_depth = best_params_xgb['max_depth'],learning_rate = best_params_xgb['learning_rate'], subsample = best_params_xgb['subsample'],colsample_bytree = best_params_xgb['colsample_bytree'],min_child_weight = best_params_xgb['min_child_weight'],gamma = best_params_xgb['gamma'])

In [20]:
##### voting classifier ###########

voting = VotingClassifier(estimators= [('lr',lr),('dt',dt),('rf',rf),('xgb',xgb)],voting = 'soft',verbose=1 )
voting.fit(x_train,y_train)
predictions = voting.predict(x_test)
accuracy = accuracy_score(y_test,predictions)
print(f"Accuracy:{accuracy}")

[Voting] ....................... (1 of 4) Processing lr, total=  42.0s
[Voting] ....................... (2 of 4) Processing dt, total= 1.1min
[Voting] ....................... (3 of 4) Processing rf, total=35.2min
[Voting] ...................... (4 of 4) Processing xgb, total=  18.5s
Accuracy:0.9996594694219257


In [21]:
#### Stacking Classifier ##########

base_models = [('dt', dt), ('rf', rf), ('xgb', xgb)]
lrm = lr 
stacking = StackingClassifier(estimators=base_models, final_estimator=lrm, verbose=1)
stacking.fit(x_train, y_train)
predictions = stacking.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9994975864240413
